# NWB File Explores

Adapted from: https://pynwb.readthedocs.io/en/stable/tutorials/general/file.html

In [1]:
import os
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np

from pynwb import NWBFile, NWBHDF5IO, TimeSeries

In [2]:
# Get local timezone
timezone = tzlocal() 

# Set time zone information
start_time = datetime(2021, 3, 20, tzinfo=tzlocal())
create_date = datetime(2021, 3, 20, tzinfo=tzlocal())

In [3]:
# Initialize an NWB file
nwbfile = NWBFile(session_description='NWBFile test',
                  identifier='NWB123',
                  session_start_time=start_time,
                  file_create_date=create_date)

In [4]:
# Dummy test data
data = np.arange(100, 200, 10)
timestamps = np.arange(10.)

#data = list(range(100, 200, 10))
#timestamps = list(range(10))

In [5]:
# Create dummy time series object
test_ts = TimeSeries(name='test_timeseries', data=data, unit='m', timestamps=timestamps)

In [6]:
# Add the dummy time series as an acquisition element
nwbfile.add_acquisition(test_ts)

In [7]:
# Check the acquisition element
nwbfile.acquisition['test_timeseries']

test_timeseries pynwb.base.TimeSeries at 0x140704403092096
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
  timestamps_unit: seconds
  unit: m

In [8]:
# Get the newly added acquisition
test_acq = nwbfile.get_acquisition('test_timeseries')

### File I/O

#### Save out NWB File

In [9]:
# Save out an example NWB file
with NWBHDF5IO('example_file_path.nwb', 'w') as io:
    io.write(nwbfile)

In [10]:
# Check for saved out NWB files
nwb_files = [file for file in os.listdir('.') if file.split('.')[-1] == 'nwb']
print(nwb_files)

['example_file_path.nwb']


In [11]:
# Run command line tool to validate NWB file
!python -m pynwb.validate example_file_path.nwb

Validating example_file_path.nwb against cached namespace information using namespace core.
 - no errors found.


#### Read in NWB File

In [12]:
# Read in an example NWB file
io = NWBHDF5IO('example_file_path.nwb', 'r')
nwbfile_in = io.read()

In [13]:
nwbfile_in

root pynwb.file.NWBFile at 0x140704444218144
Fields:
  acquisition: {
    test_timeseries <class 'pynwb.base.TimeSeries'>
  }
  file_create_date: [datetime.datetime(2021, 3, 20, 0, 0, tzinfo=tzoffset(None, -14400))]
  identifier: NWB123
  session_description: NWBFile test
  session_start_time: 2021-03-20 00:00:00-04:00
  timestamps_reference_time: 2021-03-20 00:00:00-04:00

In [14]:
test_ts_in = nwbfile_in.acquisition['test_timeseries']

In [15]:
print(test_ts_in)

test_timeseries pynwb.base.TimeSeries at 0x140704444218000
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (10,), type "<i8">
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (10,), type "<f8">
  timestamps_unit: seconds
  unit: m



In [16]:
# Close the file reader
io.close()